# Team Ratings

In [ ]:
# Local libraries
import Tools.ratings_utils as ru
import Tools.system_utils as sys

YEAR = 2025
FILENAME = f"Data/Seasons/data_{YEAR}.json"
TOURNAMENT_FILENAME = f"Data/Tournaments/tournament_{YEAR}.csv"
PICKS_FILENAME = f"Data/Tournament Picks/picks_{YEAR}.csv"


# Create data frame for valid teams in the current season that can be used for tournament simulation
score_df = ru.set_rating_data_frame(filename=FILENAME)

### Multiple Seasons

In [ ]:
import pandas as pd

YEARS = [2021, 2022, 2023, 2024, 2025]
# YEARS = [2021, 2022, 2023, 2024]
# YEARS = [2021, 2022, 2023]
# YEARS = [2021, 2022]


for YEAR in YEARS:
    FILENAME = f"Data/Seasons/data_{YEAR}.json"
    TOURNAMENT_FILENAME = f"Data/Tournaments/tournament_{YEAR}.csv"
    PICKS_FILENAME = f"Data/Tournament Picks/picks_{YEAR}.csv"
    
    if YEAR is YEARS[0]:
        # Create data frame for valid teams in the current season that can be used for tournament simulation
        score_df = ru.set_rating_data_frame(filename=FILENAME)
    else:
        # Concatenate
        new_season_score_df = ru.set_rating_data_frame(filename=FILENAME)
        score_df = pd.concat([score_df, new_season_score_df], ignore_index=True)
        
    
TOURNAMENT_YEAR = YEARS[-1]
FILENAME_YEARS = f"{YEARS[0]}-{TOURNAMENT_YEAR}"
PICKS_FILENAME = f"Data/Tournament Picks/picks_{FILENAME_YEARS}.csv"
RATINGS_FILENAME = f"Data/Season Ratings/data_{FILENAME_YEARS}.json"

## Chalk Baseline by Seed

In [ ]:
_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=None)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="chalk")

## Massey Ratings

In [ ]:
massey_ratings = ru.calculate_massey_ratings(score_df=score_df,
                                             debug=False)

_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=massey_ratings)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="massey")

# Colley Ratings

In [ ]:
colley_ratings = ru.calculate_colley_ratings(score_df=score_df,
                                             debug=False)

_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=colley_ratings)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="colley")

# Elo Ratings

In [ ]:
adj_elo_ratings = ru.calculate_elo_ratings(score_df=score_df,
                                           K=30,
                                           debug=False,
                                           adjust_K=True)

_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=adj_elo_ratings)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="adj_elo")

In [ ]:
elo_ratings = ru.calculate_elo_ratings(score_df=score_df,
                                       K=30,
                                       debug=False,
                                       adjust_K=False)

_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=elo_ratings)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="elo")

# SRS Ratings

In [ ]:
srs_ratings = ru.compile_srs_ratings(filename=FILENAME,
                                     debug=False)

_, _, tourney_dict = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                            ratings=srs_ratings)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="srs")

# Combined Ratings (numerical analysis)
Step through several possible weights for combination of normalized rating systems (in steps of 0.25 between -1 and 1).

#### Note: This may take a while to run.

In [ ]:
tourney_dict = ru.apply_ratings_weights_to_maximize_correct_picks(massey_ratings=massey_ratings,
                                                                  colley_ratings=colley_ratings,
                                                                  adj_elo_ratings=adj_elo_ratings,
                                                                  elo_ratings=elo_ratings,
                                                                  srs_ratings=srs_ratings,
                                                                  tournament_filename=TOURNAMENT_FILENAME)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="combined")

# Combined Ratings
Apply custom weights for each rating system against the current season tournament.

In [ ]:
MASSEY_WEIGHT = 0.625
COLLEY_WEIGHT = 0.375
ADJ_ELO_WEIGHT = -0.125
ELO_WEIGHT = 0.1875
SRS_WEIGHT = 0.3125

_, _, tourney_dict = ru.apply_custom_weights(massey_ratings=massey_ratings,
                                             colley_ratings=colley_ratings,
                                             adj_elo_ratings=adj_elo_ratings,
                                             elo_ratings=elo_ratings,
                                             srs_ratings=srs_ratings,
                                             tournament_filename=TOURNAMENT_FILENAME,
                                             massey_weight=MASSEY_WEIGHT,
                                             colley_weight=COLLEY_WEIGHT,
                                             adj_elo_weight=ADJ_ELO_WEIGHT,
                                             elo_weight=ELO_WEIGHT,
                                             srs_weight=SRS_WEIGHT)

sys.write_tournament_to_csv(tourney_dict=tourney_dict,
                            filename=PICKS_FILENAME,
                            rating_type="custom_combined")